# Root Cause Discovery

After processing, we obtained 2 datasets
1. "Interventional data": there are 59 AE samples (samples with aberrant gene expression) for which we have "ground truth" data about which gene is the true root cause. Each row is a gene and each column is a sample. 
2. "observational data": these are a few hundred samples for which we did not do any intervention. Each row is a gene and each column is a sample.

Below, each patient is analyzed in a separate cluster job. The code below contains the actual runtime script as well as helper functions that interface with a slurm cluster manager to automatically submit jobs. 

In [1]:
using RootCauseDiscovery
using DataFrames
using CSV
using DelimitedFiles
using JLD2
using Random
using ProgressMeter
using LinearAlgebra

# helper function to submit 1 job to run 1 command
function submit(command::String, ncores::Int, total_mem::Number, joblog_dir::String; 
        jobname="submit", waitfor=Int[], verbose=true)
    mem = round(Int, total_mem / ncores) # memory per core
    filename = "$jobname.sh"
    open(filename, "w") do io
        println(io, "#!/bin/bash")
        println(io, "#")
        println(io, "#SBATCH --job-name=$jobname")
        println(io, "#")
        println(io, "#SBATCH --time=168:00:00")
        println(io, "#SBATCH --cpus-per-task=$ncores")
        println(io, "#SBATCH --mem-per-cpu=$(mem)G")
        println(io, "#SBATCH --partition=candes,zihuai,qsu")
        println(io, "#SBATCH --output=$(joinpath(joblog_dir, "slurm-%j.out"))")
        println(io, "")
        println(io, "#save job info on joblog:")
        println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
        println(io, "")
        println(io, "# load the job environment:")
        println(io, "module load julia/1.10")
        println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
        println(io, "")
        println(io, "# run code")
        println(io, "echo \"$command\"")
        println(io, "$command")
        println(io, "")
        println(io, "#echo job info on joblog:")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
        println(io, "#echo \" \"")
    end
    # submit job and capture job ID
    io = IOBuffer()
    if length(waitfor) != 0
        run(pipeline(`sbatch --dependency=afterok:$(join(waitfor, ':')) $filename`; stdout=io))
    else
        run(pipeline(`sbatch $filename`; stdout=io))
    end
    msg = String(take!(io))
    verbose && println(stdout, msg)
    jobid = parse(Int, strip(msg)[21:end])
    # clean up and return job ID
    close(io)
    rm(filename, force=true)
    return jobid
end

# helper function to submit 1 job to run multiple commands
function submit(commands::Vector{String}, ncores::Int, 
        total_mem::Number, joblog_dir::String; 
        jobname="submit", waitfor=Int[], verbose=true)
    mem = round(Int, total_mem / ncores) # memory per core
    filename = "$jobname.sh"
    open(filename, "w") do io
        println(io, "#!/bin/bash")
        println(io, "#")
        println(io, "#SBATCH --job-name=$jobname")
        println(io, "#")
        println(io, "#SBATCH --time=168:00:00")
        println(io, "#SBATCH --cpus-per-task=$ncores")
        println(io, "#SBATCH --mem-per-cpu=$(mem)G")
        println(io, "#SBATCH --partition=candes,zihuai,qsu")
        println(io, "#SBATCH --output=$(joinpath(joblog_dir, "slurm-%j.out"))")
        println(io, "")
        println(io, "#save job info on joblog:")
        println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
        println(io, "")
        println(io, "# load the job environment:")
        println(io, "module load julia/1.10")
        println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
        println(io, "")
        for command in commands
            println(io, "echo \"$command\"")
            println(io, "$command")
        end
        println(io, "")
        println(io, "#echo job info on joblog:")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
        println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
        println(io, "#echo \" \"")
    end
    # submit job and capture job ID
    io = IOBuffer()
    if length(waitfor) != 0
        run(pipeline(`sbatch --dependency=afterok:$(join(waitfor, ':')) $filename`; stdout=io))
    else
        run(pipeline(`sbatch $filename`; stdout=io))
    end
    msg = String(take!(io))
    verbose && println(stdout, msg)
    jobid = parse(Int, strip(msg)[21:end])
    # clean up and return job ID
    close(io)
    rm(filename, force=true)
    return jobid
end

# helper function to read data
function read_data(low_count, threshold, max_cor, concatenate, dir)
    isdir(dir) || mkpath(dir)
    trainsform_int_file = joinpath(dir, "transform_int_$concatenate")
    transform_obs_file = joinpath(dir, "transform_obs_$concatenate")
    ground_truth_file = joinpath(dir, "ground_truth_$concatenate")

    if !(isfile(trainsform_int_file) && isfile(transform_obs_file) && isfile(ground_truth_file))
        # process raw data & save
        transform_int, transform_obs, ground_truth = QC_gene_expression_data(;
            low_count = low_count, 
            threshold = threshold, 
            max_cor = max_cor, 
            concatenate = concatenate
        )
        CSV.write(trainsform_int_file, transform_int)
        CSV.write(transform_obs_file, transform_obs)
        CSV.write(ground_truth_file, ground_truth)
    else
        transform_int = CSV.read(trainsform_int_file, DataFrame)
        transform_obs = CSV.read(transform_obs_file, DataFrame)
        ground_truth = CSV.read(ground_truth_file, DataFrame)
    end

    # numeric matrices
    Xobs = transform_obs[:, 2:end] |> Matrix |> transpose
    Xint = transform_int[:, 2:end] |> Matrix |> transpose
    
    # gene IDs
    gene_ids = transform_obs[:, 1]
    
    return Xobs, Xint, ground_truth, gene_ids
end

# helper function to process output of each patient (after running root cause discovery alg)
function aggregate_result(concatenate::String, dir;
    low_count = 10,
    threshold = 0.1,
    max_cor = 0.999,
    )
    cv_outdir = joinpath(dir, "$concatenate/cv")
    data_dir = joinpath(dir, "data")
    
    # import data
    Xobs, Xint, ground_truth, gene_ids = 
        read_data(low_count, threshold, max_cor, concatenate, data_dir)
    
    # only consider patients that is AE
    idx = findall(isone, ground_truth[!, "is AE"])
    ground_truth_filtered = ground_truth[idx, :]
    patient_name_filtered = ground_truth_filtered[!, "Patient ID"]

    # things to return
    df = DataFrame(
        "Patient Name"=>String[],
        "Z-score rank"=>Float64[],
        "Cholesky-score rank"=>Float64[],
        "RC Z-score"=>Float64[],
        "RC Cholesky-score"=>Float64[],
        "gene ID from Cholesky"=>String[],
        "gene name from ground truth"=>String[],
    )
    z_all = Vector{Float64}[]

    # compare our method against z score method
    @showprogress for (i, id) in enumerate(patient_name_filtered)
        try
            # compute z-score for current sample
            Xint_sample = Xint[i, :]
            Xobs_full = vcat(Xobs, Xint[setdiff(1:size(Xint, 1), i), :])
            z = RootCauseDiscovery.zscore(Xobs_full, Xint_sample)
            push!(z_all, z)
            
            # compute rank of z-score
            patient_root_cause_idx = ground_truth_filtered[i, "root cause row index in genecounts"]
            patient_root_cause_zscore = z[patient_root_cause_idx]
            root_cause_zscore_rank = count(x -> x > patient_root_cause_zscore, z)

            # cv result
            file = joinpath(cv_outdir, "$id.csv")
            cholesky_score = readdlm(file)
            root_cause_cholesky_score = cholesky_score[patient_root_cause_idx]
            root_cause_chol_rank_cv = count(x -> x > root_cause_cholesky_score, cholesky_score)
            max_chol_idx = findmax(cholesky_score)[2]
            
            push!(df, [id, root_cause_zscore_rank, 
                            root_cause_chol_rank_cv, 
                            round(patient_root_cause_zscore, digits=3), 
                            round(root_cause_cholesky_score, digits=3),
                            gene_ids[max_chol_idx],
                            ground_truth_filtered[i, "Genetic diagnosis"]
                       ])
        catch e
            println(e)
            continue
        end
    end

    # @show length(cholesky_score)
    df = df[!, [1, 3, 2, 4, 5, 6, 7]]
    sort!(df, "Cholesky-score rank")
    return df, z_all
end

Precompiling RootCauseDiscovery
  ✓ PtrArrays
  ✓ InverseFunctions
  ✓ Rmath_jll
  ✓ QuadGK
  ✓ AliasTables
  ✓ OpenSpecFun_jll
  ✓ glmnet_jll
  ✓ FillArrays
  ✓ LogExpFunctions → LogExpFunctionsInverseFunctionsExt
  ✓ Rmath
  ✓ FillArrays → FillArraysPDMatsExt
  ✓ FillArrays → FillArraysStatisticsExt
  ✓ SpecialFunctions
  ✓ SpecialFunctions → SpecialFunctionsChainRulesCoreExt
InverseFunctionsTestExt Waiting for background task / IO / timer.
[pid 30827] waiting for IO to finish:
 Handle type        uv_handle_t->data
 fs_event           0x2916bf0->0x7ff6cbfeff10
 timer              0x2910970->0x7ff6cbfeff40
This means that a package has started a background task or event source that has not finished running. For precompilation to complete successfully, the event source needs to be closed explicitly. See the developer documentation on fixing precompilation hangs for more help.
  ✓ InverseFunctions → InverseFunctionsTestExt
  ✓ StatsFuns
  ✓ Distributions
  ✓ Distributions → Distribution

aggregate_result (generic function with 1 method)

## Read processed data

In [2]:
low_count = 10
threshold = 0.1
max_cor = 0.999
concatenate = "all"
dir = "/scratch/users/bbchu/RootCauseDiscovery/8.30.2024/data"
Xobs, Xint, ground_truth, gene_ids = 
    read_data(low_count, threshold, max_cor, concatenate, dir)

([6.905019519912375 7.257078563799141 … -0.06477114998921524 5.752340009973989; 7.02426909562131 7.246401013060676 … 0.12757476399859816 5.769481834936712; … ; 6.839604518578518 7.29331359490887 … 2.7416948901919214 5.792997014900723; 6.634964406531482 7.155258338398599 … 1.9075765878191422 5.778777598727033], [7.16242663712475 7.523928621811798 … 0.8758080027262863 5.863970440970483; 7.282548459614407 7.484357868453626 … -0.04128210658791 6.128328625903546; … ; 6.954662707932615 7.206785478347909 … 2.43723143625253 5.900881832561235; 6.746641518634209 7.114730365387739 … 2.2045294686132544 5.5945535496772845], 70×6 DataFrame
 Row │ Patient ID  Genetic diagnosis  gene_id          patient column index in ⋯
     │ String7     String15           String15         Int64                   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ R62943      C19orf70           ENSG00000174917                          ⋯
   2 │ R98254      MRPL38             ENSG

## Submit main jobs (1 job for each patient)

Below first shows the actual runtime script

In [ ]:
# put this codeblock in script /scratch/users/bbchu/RootCauseDiscovery/run_realData_RCD.jl

using RootCauseDiscovery
using DataFrames
using CSV
using DelimitedFiles
using JLD2
using Random
using LinearAlgebra
BLAS.set_num_threads(1)

# inputs to the script
patient_id = ARGS[1]
y_idx_z_threshold = parse(Float64, ARGS[2])
outfile = ARGS[3]
low_count = parse(Int, ARGS[4]) # min number of gene count exceeding `threshold` is needed to prevent gene from being filtered out
threshold = parse(Float64, ARGS[5]) # for processing raw data
seed = parse(Int, ARGS[6])
nshuffles = parse(Int, ARGS[7])
method = ARGS[8] # "cv" or "nhalf"
max_cor = parse(Float64, ARGS[9])
concatenate = ARGS[10] # "all", "ns", or "ss"

# for testing
# patient_id = "R62943"
# y_idx_z_threshold = 1.5
# outfile = "fdsa"
# low_count = 10
# threshold = 0.1
# seed = 1111
# nshuffles = 1
# method = "cv"
# max_cor = 0.999
# concatenate = "all"

# process raw data
transform_int, transform_obs, _ = 
    QC_gene_expression_data(low_count = low_count, 
    threshold = threshold, max_cor = max_cor, 
    concatenate = concatenate);

# numeric matrices
Xobs = transform_obs[:, 2:end] |> Matrix |> transpose
Xint = transform_int[:, 2:end] |> Matrix |> transpose
i = findfirst(x -> x == patient_id, names(transform_int)[2:end])
if !isnothing(i)
    Xint_sample = Xint[i, :]

    # concat Xobs
    nint = size(Xint, 1)
    Xobs_full = vcat(Xobs, Xint[setdiff(1:nint, i), :])

    # run main alg
    Random.seed!(seed)
    @time root_cause_score = root_cause_discovery_high_dimensional(
        Xobs_full, Xint_sample, method, y_idx_z_threshold=y_idx_z_threshold,
        nshuffles=nshuffles
    );

    # save result
    writedlm(outfile, root_cause_score)
else
    println("patient $patient_id not found in interventional samples!")
end

println("Done!")

### Submit jobs for each patient

A typical patient requires 10-48h to run with 24 cores

In [4]:
julia_exe = "/scratch/users/bbchu/RootCauseDiscovery/run_realData_RCD_ines.jl"
dirs = [
    "/scratch/users/bbchu/RootCauseDiscovery/8.30.2024"
]
joblog_dirs = [joinpath(dirs[i], "joblogs") for i in eachindex(dirs)]
mkpath.(joblog_dirs)
seeds = [8302024]
low_counts = [10]
thresholds = [0.1]
nshuffles = 30
y_idx_z_threshold = 1.5
method = "cv"
max_cor = 0.999
concatenates = ["all"]

# patient IDs
patient_name = ["R62943", "R98254", "R89912", "R19100", "R15264", "R36605", "R61100", 
    "R77611", "R16472", "R51757", "R80346", "R20754", "R25473", "R28774", "R96820", 
    "R21147", "R64921", "R52016", "R46723", "R80184", "R59185", "R44456", "R24289", 
    "R91273", "R60537", "R72253", "R75000", "R91016", "R82353", "R34834", "R78764", 
    "R30367", "R76358", "R12128", "R45867", "R31640", "R95723", "R55237", "R18626", 
    "R34820", "R25912", "R11258", "R64046", "R54158", "R19907", "R27473", "R70186", 
    "R26710", "R30525", "R29620", "R64055", "R15748", "R66814", "R77365", "R21993", 
    "R42505", "R64948", "R21470", "R47816"]

# for testing
# nshuffles = 2
# y_idx_z_threshold = 20
# patient_name = ["R62943", "R98254"]

# submit jobs
for concat in concatenates
    for (dir, joblog_dir, seed, low_count, threshold) in zip(dirs, joblog_dirs, seeds, low_counts, thresholds)
        for patient in patient_name
            outdir = joinpath(dir, concat, method)
            isdir(outdir) || mkpath(outdir)

            outfile = joinpath(outdir, "$(patient).csv")
            if !isfile(outfile)
                cmd = "julia -t 24 $julia_exe $patient $y_idx_z_threshold $outfile $low_count $threshold $seed $nshuffles $method $max_cor $concat"
                submit(cmd, 24, 64, joblog_dir, jobname=patient)
            end
        end
    end
end

Submitted batch job 53850521



# Plot result

### 8.30.2024

In [3]:
dir = "/scratch/users/bbchu/RootCauseDiscovery/8.30.2024"
df_all, z_all = aggregate_result("all", dir);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:12


In [4]:
display(df_all)

Row,Patient Name,Cholesky-score rank,Z-score rank,RC Z-score,RC Cholesky-score,gene ID from Cholesky,gene name from ground truth
,String,Float64,Float64,Float64,Float64,String,String
1,R19100,0.0,0.0,24.275,8.501,ENSG00000151729,SLC25A4
2,R61100,0.0,1.0,33.222,21.201,ENSG00000163382,NAXE
3,R77611,0.0,9.0,8.15,12.217,ENSG00000091140,DLD
4,R16472,0.0,0.0,39.138,11.533,ENSG00000131368,MRPS25
5,R28774,0.0,0.0,133.791,10.542,ENSG00000113845,TIMMDC1
6,R64921,0.0,2.0,17.351,15.255,ENSG00000090674,MCOLN1
7,R80184,0.0,10.0,11.534,8.062,ENSG00000059573,ALDH18A1
8,R59185,0.0,3056.0,4.16,8.944,ENSG00000164258,NDUFS4
9,R91273,0.0,1.0,12.237,10.999,ENSG00000171503,ETFDH


## Save result

In [8]:
# final ranking
CSV.write("/scratch/users/bbchu/RootCauseDiscovery/8.30.2024/result_all.csv", df_all)

"/scratch/users/bbchu/RootCauseDiscovery/8.30.2024/result_all.csv"

Also save Z-scores

In [9]:
writedlm("/scratch/users/bbchu/RootCauseDiscovery/8.30.2024/zscores_all.csv", hcat(z_all...)')